In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from pathlib import Path
cudnn.benchmark = True
plt.ion() 
from tqdm import tqdm
import torchvision.models as models
from collections import Counter
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import warnings
from sklearn.exceptions import UndefinedMetricWarning
import cv2


In [ ]:
# Data normalization for training
data_transforms = {
    'Train_sorted': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Validation_sorted': transforms.Compose([
        transforms.Resize((256, 256)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test_sorted': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
}

data_dir = '/Users/inescocco/Desktop/ISIC2019'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['Train_sorted', 'Validation_sorted']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=0)
              for x in ['Train_sorted', 'Validation_sorted']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['Train_sorted', 'Validation_sorted']}
class_names = image_datasets['Train_sorted'].classes
print(class_names)


# GradCAM

In [ ]:
def compute_gradCAM(model, folder_path, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),  
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255 
        
        # Ensuring the heatmap and image are in the same color format (RGB)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB) 
        
        # Overlaying heatmap on original image with weighted combination
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  # Convert the result to uint8
    
    for folder in class_names:
        # Constructing the path for each class folder
        train_folder_path = os.path.join(folder_path, folder)
        
        # Ensuring it's a directory (skip files)
        if not os.path.isdir(train_folder_path):
            continue

        # Getting the first image in the folder
        images = os.listdir(train_folder_path)
        if len(images) > 0:
            img_path = os.path.join(train_folder_path, images[2])
            print(f"Processing {img_path}")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')

            # Preprocessing the image
            input_tensor = transform(img_pil)
            input_tensor = input_tensor.unsqueeze(0).to(device)

            # Setting model to evaluation mode
            model.eval()

            # 1. Defining a variable to store the activations of the chosen layer
            middle_layer_activations = None

            # 2. Defining a hook function
            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output
                middle_layer_activations.requires_grad_(True)
                middle_layer_activations.retain_grad()

            
            # TARGET LAYER
            
            # 3. Forward pass to get prediction and activations TARGET LAYER
            target_layer = model.features[-1] 
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            # 4. Getting predicted class (for Grad-CAM)
            predicted_class = torch.argmax(output, dim=1).item()

            # Mapping the predicted class index to the actual class name
            predicted_class_name = class_names[predicted_class]
            correct_status = "(Correct)" if predicted_class_name == folder else "(Incorrect)"
            print(f"Predicted class: {predicted_class_name} {correct_status}")

            # Calculating gradient
            output[0, predicted_class].backward()

            # Getting the gradients and activations
            gradients = middle_layer_activations.grad.data
            pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

            # 5. Weight activations by gradients to get the heatmap
            activations = middle_layer_activations.detach().cpu().numpy()[0]
            for i in range(activations.shape[0]):
                activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
            heatmap = np.mean(activations, axis=0)

            # 6. Normalise and resize the heatmap
            heatmap = np.maximum(heatmap, 0) 
            if np.max(heatmap) != 0:
                heatmap /= np.max(heatmap)

            heatmap = cv2.resize(heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)


            # 7. Overlay the heatmap on the original image using a weighted combination
            img = np.array(img_pil, dtype=np.float32) / 255.0  
            visualization = show_cam_on_image(img, heatmap, intensity=0.5) 

            # 8. Display original image and Grad-CAM visualization side by side
            fig, axs = plt.subplots(1, 2, figsize=(8, 4))

            # Original Image
            axs[0].imshow(np.array(img_pil)) 
            axs[0].text(0.5, 1.1, images[0], size=12, ha="center", transform=axs[0].transAxes)
            axs[0].set_title(f"Original Image ({folder})" , fontsize=14)
            axs[0].axis('off')

            # Grad-CAM Visualization
            axs[1].imshow(visualization)
            axs[1].set_title(f"Grad-CAM: {predicted_class_name} {correct_status}", fontsize=14)
            axs[1].axis('off')

            plt.show()


In [ ]:
def compute_gradCAM(model_path, folder_path, class_names, selected_folder):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resizing image to model input size
        transforms.ToTensor(),  # Converting the image to tensor and scales it [0, 1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  # Normalising heatmap to [0, 1]
        
        # Ensuring the heatmap and image are in the same color format (RGB)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  # Converting BGR to RGB
        
        # Overlaying heatmap on original image with weighted combination
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  # Converting the result to uint8
    
    # Ensuring that the folder exists
    selected_folder_path = os.path.join(folder_path, selected_folder)
    
    if not os.path.isdir(selected_folder_path):
        print(f"Folder {selected_folder} does not exist in the given path.")
        return
    
    # Getting all the images in the selected folder
    images = os.listdir(selected_folder_path)
    if len(images) == 0:
        print(f"No images found in {selected_folder}.")
        return

    for img_name in images:
        img_path = os.path.join(selected_folder_path, img_name)
        print(f"Processing {img_path}")
        print('Actual class:', selected_folder)

        img_pil = Image.open(img_path).convert('RGB')

        # Preprocessing the image
        input_tensor = transform(img_pil)
        input_tensor = input_tensor.unsqueeze(0).to(device)

        # Setting model to evaluation mode
        model.eval()

        # 1. Define a variable to store the activations of the chosen layer
        middle_layer_activations = None

        # 2. Define a hook function
        def hook_fn(module, input, output):
            nonlocal middle_layer_activations
            middle_layer_activations = output
            middle_layer_activations.requires_grad_(True)
            middle_layer_activations.retain_grad()

        # TARGET LAYER
        target_layer = model.features[-1] 
        hook = target_layer.register_forward_hook(hook_fn)
        output = model(input_tensor)

        # 4. Get predicted class (for Grad-CAM)
        predicted_class = torch.argmax(output, dim=1).item()

        # Map the predicted class index to the actual class name
        predicted_class_name = class_names[predicted_class]
        correct_status = "(Correct)" if predicted_class_name == selected_folder else "(Incorrect)"
        print(f"Predicted class: {predicted_class_name} {correct_status}")

        # Calculate gradient
        output[0, predicted_class].backward()

        # Get the gradients and activations
        gradients = middle_layer_activations.grad.data
        pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

        # 5. Weight activations by gradients to get the heatmap
        activations = middle_layer_activations.detach().cpu().numpy()[0]
        for i in range(activations.shape[0]):
            activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
        heatmap = np.mean(activations, axis=0)

        # 6. Normalize and resize the heatmap
        heatmap = np.maximum(heatmap, 0)  # Ensure non-negative values
        if np.max(heatmap) != 0:
            heatmap /= np.max(heatmap)

        # Resizing heatmap to match image size
        heatmap = cv2.resize(heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

        # 7. Overlay the heatmap on the original image using a weighted combination
        img = np.array(img_pil, dtype=np.float32) / 255.0  # Normalising image
        visualization = show_cam_on_image(img, heatmap, intensity=0.5)

        # 8. Display original image and Grad-CAM visualization side by side
        fig, axs = plt.subplots(1, 2, figsize=(7, 5))

        # Original Image
        axs[0].imshow(np.array(img_pil))  # Converting the PIL image to numpy for plotting
        axs[0].set_title(f"Original Image ({selected_folder})", fontsize=14)
        axs[0].axis('off')

        # Grad-CAM Visualization
        axs[1].imshow(visualization)
        axs[1].set_title(f"Grad-CAM: {predicted_class_name} {correct_status}", fontsize=14)
        axs[1].axis('off')

        plt.show()


In [ ]:
model_path = "model_b7_2_epoch_7_3.pth"
folder_path = '/Users/inescocco/Desktop/ISIC2019/Test_sorted'
compute_gradCAM(model_path, folder_path, class_names)

In [ ]:
compute_gradCAM(model_path, folder_path, class_names, 'NV')

# Contrastive GradCAM

In [ ]:
def contrastive_gradCAM(model, folder_path, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resizing image to model input size
        transforms.ToTensor(),  # Converting the image to tensor and scales it [0, 1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  # Normalize heatmap to [0, 1]
        
        # Ensuring the heatmap and image are in the same color format (RGB)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        
        # Overlaying heatmap on original image with weighted combination
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam) 
    
    for folder in class_names:
        # Constructing the path for each class folder
        train_folder_path = os.path.join(folder_path, folder)
        
        # Ensuring it's a directory (skip files)
        if not os.path.isdir(train_folder_path):
            continue

        # Getting the first image in the folder
        images = os.listdir(train_folder_path)[2:]
        if len(images) > 0:
            img_path = os.path.join(train_folder_path, images[0])
            print(f"Processing {img_path}")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')

            # Preprocessing the image
            input_tensor = transform(img_pil)
            input_tensor = input_tensor.unsqueeze(0).to(device)
            model.eval()

            # 1. Define a variable to store the activations of the chosen layer
            middle_layer_activations = None

            # 2. Define a hook function
            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output
                middle_layer_activations.requires_grad_(True)
                middle_layer_activations.retain_grad()

            # TARGET LAYER
            target_layer = model.features[-1] 
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            # 4. Get predicted class (for Grad-CAM)
            predicted_class = torch.argmax(output, dim=1).item()

            # Map the predicted class index to the actual class name
            predicted_class_name = class_names[predicted_class]
            if predicted_class_name == folder:
                print (f'Predicted class: {predicted_class_name}')
            if predicted_class_name != folder: 
                correct_status = "(Incorrect)"
                print(f"Predicted class: {predicted_class_name}{correct_status}")

                # Calculate gradient for predicted class
                output[0, predicted_class].backward(retain_graph=True)

                # Get the gradients and activations for predicted class
                gradients = middle_layer_activations.grad.data
                pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

                # 5. Weight activations by gradients to get the heatmap for predicted class
                activations = middle_layer_activations.detach().cpu().numpy()[0]
                for i in range(activations.shape[0]):
                    activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
                heatmap_pred = np.mean(activations, axis=0)

                # 6. Normalising and resize the heatmap for predicted class
                heatmap_pred = np.maximum(heatmap_pred, 0)
                if np.max(heatmap_pred) != 0:
                    heatmap_pred /= np.max(heatmap_pred)
                heatmap_pred = cv2.resize(heatmap_pred, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

                # Calculating gradient for true class (for contrastive heatmap)
                true_class = class_names.index(folder)
                output[0, true_class].backward(retain_graph=True)  

                # Getting the gradients and activations for true class
                gradients = middle_layer_activations.grad.data
                pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

                # 7. Weight activations by gradients to get the heatmap for true class
                activations = middle_layer_activations.detach().cpu().numpy()[0]
                for i in range(activations.shape[0]):
                    activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
                heatmap_true = np.mean(activations, axis=0)

                # 8. Normalising and resize the heatmap for true class
                heatmap_true = np.maximum(heatmap_true, 0)
                if np.max(heatmap_true) != 0:
                    heatmap_true /= np.max(heatmap_true)
                heatmap_true = cv2.resize(heatmap_true, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

                # 9. Contrastive heatmap
                contrastive_heatmap = heatmap_pred - heatmap_true

                # 10. Overlaying the heatmap on the original image using a weighted combination
                img = np.array(img_pil, dtype=np.float32) / 255.0  # Normalize image
                visualization_pred = show_cam_on_image(img, heatmap_pred, intensity=0.5)
                visualization_true = show_cam_on_image(img, heatmap_true, intensity=0.5)
                visualization_contrastive = show_cam_on_image(img, contrastive_heatmap, intensity=0.5)

                # 11. Displaying original image, predicted heatmap, true heatmap, and contrastive heatmap side by side
                fig, axs = plt.subplots(1, 4, figsize=(15, 7))

                # Original Image
                axs[0].imshow(np.array(img_pil)) 
                axs[0].set_title(f"Original Image ({folder})"  , fontsize=12)
                axs[0].axis('off')

                # Predicted Class Heatmap
                axs[1].imshow(visualization_pred)
                axs[1].set_title(f"Grad-CAM Pred Label: {predicted_class_name}", fontsize=12)
                axs[1].axis('off')

                # True Class Heatmap
                axs[2].imshow(visualization_true)
                axs[2].set_title(f"Grad-CAM True Label: {folder}", fontsize=12)
                axs[2].axis('off')

                # Contrastive Heatmap
                axs[3].imshow(visualization_contrastive)
                axs[3].set_title("GradCAMs Difference", fontsize=12)
                axs[3].axis('off')

                plt.show()


In [ ]:
def contrastive_gradCAM(model, folder_path, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize image to model input size
        transforms.ToTensor(),  # Converts the image to tensor and scales it [0, 1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255 
        
        # Ensuring the heatmap and image are in the same color format (RGB)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  # Converting BGR to RGB
        
        # Overlaying heatmap on original image with weighted combination
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  # Converting the result to uint8
    
    for folder in class_names:
        # Constructing the path for each class folder
        train_folder_path = os.path.join(folder_path, folder)
        
        # Ensuring it's a directory (skip files)
        if not os.path.isdir(train_folder_path):
            continue

        # Getting the images in the folder
        images = os.listdir(train_folder_path)
        for img_name in images:
            img_path = os.path.join(train_folder_path, img_name)
            print(f"Processing {img_path}")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')

            # Preprocessing the image
            input_tensor = transform(img_pil)
            input_tensor = input_tensor.unsqueeze(0).to(device)

            # Setting model to evaluation mode
            model.eval()

            # 1. Define a variable to store the activations of the chosen layer
            middle_layer_activations = None

            # 2. Define a hook function
            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output
                middle_layer_activations.requires_grad_(True)
                middle_layer_activations.retain_grad()

            # TARGET LAYER
            target_layer = model.features[-1] 
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            # 4. Get predicted class (for Grad-CAM)
            predicted_class = torch.argmax(output, dim=1).item()

            # Map the predicted class index to the actual class name
            predicted_class_name = class_names[predicted_class]

            # Checking if the prediction is correct
            if predicted_class_name == folder:
                print(f'Predicted class: {predicted_class_name} (Correct)')
                continue  # Move to the next image if the prediction is correct
            else:
                correct_status = "(Incorrect)"
                print(f"Predicted class: {predicted_class_name} {correct_status}")

                # Calculating gradient for predicted class
                output[0, predicted_class].backward(retain_graph=True)

                # Getting the gradients and activations for predicted class
                gradients = middle_layer_activations.grad.data
                pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

                # 5. Weight activations by gradients to get the heatmap for predicted class
                activations = middle_layer_activations.detach().cpu().numpy()[0]
                for i in range(activations.shape[0]):
                    activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
                heatmap_pred = np.mean(activations, axis=0)

                # 6. Normalise and resize the heatmap for predicted class
                heatmap_pred = np.maximum(heatmap_pred, 0)
                if np.max(heatmap_pred) != 0:
                    heatmap_pred /= np.max(heatmap_pred)
                heatmap_pred = cv2.resize(heatmap_pred, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

                # Calculating gradient for true class (for contrastive heatmap)
                true_class = class_names.index(folder)
                output[0, true_class].backward(retain_graph=True)  

                # Getting the gradients and activations for true class
                gradients = middle_layer_activations.grad.data
                pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

                # 7. Weight activations by gradients to get the heatmap for true class
                activations = middle_layer_activations.detach().cpu().numpy()[0]
                for i in range(activations.shape[0]):
                    activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
                heatmap_true = np.mean(activations, axis=0)

                # 8. Normalising and resize the heatmap for true class
                heatmap_true = np.maximum(heatmap_true, 0)
                if np.max(heatmap_true) != 0:
                    heatmap_true /= np.max(heatmap_true)
                heatmap_true = cv2.resize(heatmap_true, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

                # 9. Contrastive heatmap
                contrastive_heatmap = heatmap_pred - heatmap_true

                # 10. Overlaying the heatmap on the original image using a weighted combination
                img = np.array(img_pil, dtype=np.float32) / 255.0  # Normalize image
                visualization_pred = show_cam_on_image(img, heatmap_pred, intensity=0.5)
                visualization_true = show_cam_on_image(img, heatmap_true, intensity=0.5)
                visualization_contrastive = show_cam_on_image(img, contrastive_heatmap, intensity=0.5)

                # 11. Displaying original image, predicted heatmap, true heatmap, and contrastive heatmap side by side
                fig, axs = plt.subplots(1, 4, figsize=(15, 7))

                # Original Image
                axs[0].imshow(np.array(img_pil))
                axs[0].set_title(f"Original Image ({folder})"  , fontsize=14)
                axs[0].axis('off')

                # Predicted Class Heatmap
                axs[1].imshow(visualization_pred)
                axs[1].set_title(f"Grad-CAM Pred Label: {predicted_class_name}", fontsize=14)
                axs[1].axis('off')

                # True Class Heatmap
                axs[2].imshow(visualization_true)
                axs[2].set_title(f"Grad-CAM True Label: {folder}", fontsize=14)
                axs[2].axis('off')

                # Contrastive Heatmap
                axs[3].imshow(visualization_contrastive)
                axs[3].set_title("GradCAMs Difference", fontsize=14)
                axs[3].axis('off')

                plt.show()

                break




In [ ]:
def contrastive_gradCAM(model, folder_path, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resizing image to model input size
        transforms.ToTensor(),  # Converting the image to tensor and scales it [0, 1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  # Normalising heatmap to [0, 1]
        
        # Ensuring the heatmap and image are in the same color format (RGB)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB) 
        
        # Overlaying heatmap on original image with weighted combination
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  # Converting the result to uint8
    
    for folder in class_names:
        # Constructing the path for each class folder
        train_folder_path = os.path.join(folder_path, folder)
        
        # Ensuring it's a directory (skip files)
        if not os.path.isdir(train_folder_path):
            continue

        # Getting the images in the folder
        images = os.listdir(train_folder_path)
        for img_name in images:
            img_path = os.path.join(train_folder_path, img_name)
            print(f"Processing {img_path}")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')

            # Preprocessing the image
            input_tensor = transform(img_pil)
            input_tensor = input_tensor.unsqueeze(0).to(device)

            # Setting model to evaluation mode
            model.eval()

            # 1. Define a variable to store the activations of the chosen layer
            middle_layer_activations = None

            # 2. Define a hook function
            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output
                middle_layer_activations.requires_grad_(True)
                middle_layer_activations.retain_grad()

            # TARGET LAYER
            target_layer = model.features[-1] 
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            # 4. Get predicted class (for Grad-CAM)
            predicted_class = torch.argmax(output, dim=1).item()

            # Map the predicted class index to the actual class name
            predicted_class_name = class_names[predicted_class]

            # Checking if the prediction is correct
            if predicted_class_name == folder:
                print(f'Predicted class: {predicted_class_name} (Correct)')
                continue 

            # If the prediction is incorrect, process the heatmaps
            correct_status = "(Incorrect)"
            print(f"Predicted class: {predicted_class_name} {correct_status}")

            # Calculating gradient for predicted class
            output[0, predicted_class].backward(retain_graph=True)

            # Getting the gradients and activations for predicted class
            gradients = middle_layer_activations.grad.data
            pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

            # 5. Weight activations by gradients to get the heatmap for predicted class
            activations = middle_layer_activations.detach().cpu().numpy()[0]
            for i in range(activations.shape[0]):
                activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
            heatmap_pred = np.mean(activations, axis=0)

            # 6. Normalise and resize the heatmap for predicted class
            heatmap_pred = np.maximum(heatmap_pred, 0)
            if np.max(heatmap_pred) != 0:
                heatmap_pred /= np.max(heatmap_pred)
            heatmap_pred = cv2.resize(heatmap_pred, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

            # Calculating gradient for true class (for contrastive heatmap)
            true_class = class_names.index(folder)
            output[0, true_class].backward(retain_graph=True) 

            # Getting the gradients and activations for true class
            gradients = middle_layer_activations.grad.data
            pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

            # 7. Weight activations by gradients to get the heatmap for true class
            activations = middle_layer_activations.detach().cpu().numpy()[0]
            for i in range(activations.shape[0]):
                activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
            heatmap_true = np.mean(activations, axis=0)

            # 8. Normalise and resize the heatmap for true class
            heatmap_true = np.maximum(heatmap_true, 0)
            if np.max(heatmap_true) != 0:
                heatmap_true /= np.max(heatmap_true)
            heatmap_true = cv2.resize(heatmap_true, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

            # 9. Contrastive heatmap
            contrastive_heatmap = heatmap_pred - heatmap_true

            # 10. Overlay the heatmap on the original image using a weighted combination
            img = np.array(img_pil, dtype=np.float32) / 255.0 
            visualization_pred = show_cam_on_image(img, heatmap_pred, intensity=0.5)
            visualization_true = show_cam_on_image(img, heatmap_true, intensity=0.5)
            visualization_contrastive = show_cam_on_image(img, contrastive_heatmap, intensity=0.5)

            # 11. Displaying original image, predicted heatmap, true heatmap, and contrastive heatmap side by side
            fig, axs = plt.subplots(1, 4, figsize=(15, 7))

            # Original Image
            axs[0].imshow(np.array(img_pil))  # Convert the PIL image to numpy for plotting
            axs[0].set_title(f"Original Image ({folder})"  , fontsize=14)
            axs[0].axis('off')

            # Predicted Class Heatmap
            axs[1].imshow(visualization_pred)
            axs[1].set_title(f"Grad-CAM Pred Label: {predicted_class_name}", fontsize=14)
            axs[1].axis('off')

            # True Class Heatmap
            axs[2].imshow(visualization_true)
            axs[2].set_title(f"Grad-CAM True Label: {folder}", fontsize=14)
            axs[2].axis('off')

            # Contrastive Heatmap
            axs[3].imshow(visualization_contrastive)
            axs[3].set_title("GradCAMs Difference", fontsize=14)
            axs[3].axis('off')

            plt.show()

            continue


In [ ]:
contrastive_gradCAM(model_path, folder_path, class_names)

# GradCAM++

In [ ]:
def compute_gradCAMplusplus(model, folder_path, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resizing image to model input size
        transforms.ToTensor(),  # Converting the image to tensor and scales it [0, 1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  # Normalize heatmap to [0, 1]
        
        # Ensuring the heatmap and image are in the same color format (RGB)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB) 
        
        # Overlaying heatmap on original image with weighted combination
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  # Convert the result to uint8
    
    for folder in class_names:
        # Constructing the path for each class folder
        train_folder_path = os.path.join(folder_path, folder)
        
        # Ensuring it's a directory (skip files)
        if not os.path.isdir(train_folder_path):
            continue

        # Getting the first image in the folder
        images = os.listdir(train_folder_path)
        if len(images) > 0:
            img_path = os.path.join(train_folder_path, images[2])
            print(f"Processing {img_path}")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')

            # Preprocessing the image
            input_tensor = transform(img_pil)
            input_tensor = input_tensor.unsqueeze(0).to(device)

            # Setting model to evaluation mode
            model.eval()

            # 1. Define a variable to store the activations of the chosen layer
            middle_layer_activations = None

            # 2. Define a hook function
            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output
                middle_layer_activations.requires_grad_(True)
                middle_layer_activations.retain_grad()

            
            # TARGET LAYER
            # 3. Forward pass to get prediction and activations TARGET LAYER
            target_layer = model.features[-1]  
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            # 4. Get predicted class (for Grad-CAM++)
            predicted_class = torch.argmax(output, dim=1).item()

            # Mapping the predicted class index to the actual class name
            predicted_class_name = class_names[predicted_class]
            correct_status = "(Correct)" if predicted_class_name == folder else "(Incorrect)"
            print(f"Predicted class: {predicted_class_name} {correct_status}")

            # Calculating gradient
            output[0, predicted_class].backward(retain_graph=True) 

            # Getting the gradients and activations
            gradients = middle_layer_activations.grad.data
            activations = middle_layer_activations.detach().cpu().numpy()[0]

            # Grad-CAM++ specific: Use higher-order gradients
            alpha = torch.clamp(gradients, min=0)  
            beta = torch.clamp(gradients, min=0)   
            gamma = torch.clamp(gradients, min=0) 

            # Computing the weighted activations based on Grad-CAM++ formula
            weighted_activations = np.zeros_like(activations)

            # Looping over all channels
            for i in range(activations.shape[0]):
                # Sum over the spatial dimensions (height x width)
                weighted_activations[i, :, :] = activations[i, :, :] * (alpha[0, i, :, :].cpu().numpy() + 
                                                                       beta[0, i, :, :].cpu().numpy() ** 2 + 
                                                                       gamma[0, i, :, :].cpu().numpy() ** 3)

            heatmap = np.sum(weighted_activations, axis=0)

            # 5. Normalise and resize the heatmap
            heatmap = np.maximum(heatmap, 0)  # Ensure non-negative values
            if np.max(heatmap) != 0:
                heatmap /= np.max(heatmap)

            # Resizing the heatmap
            heatmap = cv2.resize(heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

            # 6. Overlay the heatmap on the original image using a weighted combination
            img = np.array(img_pil, dtype=np.float32) / 255.0 
            visualization = show_cam_on_image(img, heatmap, intensity=0.5) 

            # 7. Display original image and Grad-CAM++ visualization side by side
            fig, axs = plt.subplots(1, 2, figsize=(10, 5))

            # Original Image
            axs[0].imshow(np.array(img_pil))
            axs[0].text(0.5, 1.1, images[0], size=12, ha="center", transform=axs[0].transAxes)
            axs[0].set_title(f"Original Image ({folder})" , fontsize=14)
            axs[0].axis('off')

            # Grad-CAM++ Visualization
            axs[1].imshow(visualization)
            axs[1].set_title(f"Grad-CAM++: {predicted_class_name} {correct_status}", fontsize=14)
            axs[1].axis('off')

            plt.show()

In [ ]:
compute_gradCAMplusplus(model_path, folder_path, class_names)

## Contrastive GradCAM++

In [ ]:
def contrastive_gradCAMplusplus(model_path, folder_path, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resizing image to model input size
        transforms.ToTensor(),  # Converting the image to tensor and scales it [0, 1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation
    ])

    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  # Normalising heatmap to [0, 1]
        
        # Ensuring the heatmap and image are in the same color format (RGB)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  # Converting BGR to RGB
        
        # Overlaying heatmap on original image with weighted combination
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam) 
    
    for folder in class_names:
        # Constructing the path for each class folder
        train_folder_path = os.path.join(folder_path, folder)
        
        # Ensuring it's a directory (skip files)
        if not os.path.isdir(train_folder_path):
            continue

        # Getting the first image in the folder
        images = os.listdir(train_folder_path)
        if len(images) > 0:
            img_path = os.path.join(train_folder_path, images[2]) 
            print(f"Processing {img_path}")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')

            # Preprocessing the image
            input_tensor = transform(img_pil)
            input_tensor = input_tensor.unsqueeze(0).to(device)

            # Setting model to evaluation mode
            model.eval()

            # 1. Define a variable to store the activations of the chosen layer
            middle_layer_activations = None

            # 2. Define a hook function
            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output
                middle_layer_activations.requires_grad_(True)
                middle_layer_activations.retain_grad()

            # TARGET LAYER
            # 3. Perform the forward pass for the predicted class
            target_layer = model.features[-1]
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            # 4. Get predicted class (for Grad-CAM++)
            predicted_class = torch.argmax(output, dim=1).item()

            # Mapping the predicted class index to the actual class name
            predicted_class_name = class_names[predicted_class]
            correct_status = "(Correct)" if predicted_class_name == folder else "(Incorrect)"
            if predicted_class_name == folder:
                print(f"Predicted class: {predicted_class_name}")
            # Only print and proceed if predicted class is not the same as the true class
            if predicted_class_name != folder:
                print(f"Predicted class: {predicted_class_name}{correct_status}")

                # Back propagation for predicted class
                output[0, predicted_class].backward(retain_graph=True)  # For predicted class
                predicted_gradients = middle_layer_activations.grad.data
                predicted_activations = middle_layer_activations.detach().cpu().numpy()[0]

                # Grad-CAM++ for predicted class
                alpha = torch.clamp(predicted_gradients, min=0)
                beta = torch.clamp(predicted_gradients, min=0)
                gamma = torch.clamp(predicted_gradients, min=0)

                # Computing weighted activations for predicted class
                weighted_predicted_activations = np.zeros_like(predicted_activations)
                for i in range(predicted_activations.shape[0]):
                    weighted_predicted_activations[i, :, :] = predicted_activations[i, :, :] * (
                        alpha[0, i, :, :].cpu().numpy() +
                        beta[0, i, :, :].cpu().numpy() ** 2 +
                        gamma[0, i, :, :].cpu().numpy() ** 3
                    )

                # Summing across all channels to generate the final heatmap for predicted class
                heatmap_predicted = np.sum(weighted_predicted_activations, axis=0)

                # 5. Now perform the forward pass for the true class
                true_class = class_names.index(folder)  # Getting the true class from the folder name
                model.zero_grad()  # Zero the gradients
                output = model(input_tensor)  # Forward pass again for true class
                output[0, true_class].backward(retain_graph=True)  # Back propagation for true class

                true_gradients = middle_layer_activations.grad.data
                true_activations = middle_layer_activations.detach().cpu().numpy()[0]

                # Grad-CAM++ for true class
                alpha = torch.clamp(true_gradients, min=0)
                beta = torch.clamp(true_gradients, min=0)
                gamma = torch.clamp(true_gradients, min=0)

                # Computing weighted activations for true class
                weighted_true_activations = np.zeros_like(true_activations)
                for i in range(true_activations.shape[0]):
                    weighted_true_activations[i, :, :] = true_activations[i, :, :] * (
                        alpha[0, i, :, :].cpu().numpy() +
                        beta[0, i, :, :].cpu().numpy() ** 2 +
                        gamma[0, i, :, :].cpu().numpy() ** 3
                    )

                # Summing across all channels to generate the final heatmap for true class
                heatmap_true = np.sum(weighted_true_activations, axis=0)

                # Normalising heatmaps
                heatmap_predicted = np.maximum(heatmap_predicted, 0)
                if np.max(heatmap_predicted) != 0:
                    heatmap_predicted /= np.max(heatmap_predicted)

                heatmap_true = np.maximum(heatmap_true, 0)
                if np.max(heatmap_true) != 0:
                    heatmap_true /= np.max(heatmap_true)

                # Contrastive heatmap (difference between predicted and true heatmaps)
                contrastive_heatmap = np.abs(heatmap_predicted - heatmap_true)

                # Resizing the heatmaps
                heatmap_predicted = cv2.resize(heatmap_predicted, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)
                heatmap_true = cv2.resize(heatmap_true, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)
                contrastive_heatmap = cv2.resize(contrastive_heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

                # Overlaying the heatmaps on the original image
                img = np.array(img_pil, dtype=np.float32) / 255.0  # Normalize image
                visualization_predicted = show_cam_on_image(img, heatmap_predicted, intensity=0.6)
                visualization_true = show_cam_on_image(img, heatmap_true, intensity=0.6)
                visualization_contrastive = show_cam_on_image(img, contrastive_heatmap, intensity=0.6)

                # Displaying the results
                fig, axs = plt.subplots(1, 4, figsize=(20, 5))

                # Original Image
                axs[0].imshow(np.array(img_pil))  # Convert the PIL image to numpy for plotting
                axs[0].text(0.5, 1.1, images[0], size=12, ha="center", transform=axs[0].transAxes)
                axs[0].set_title("Original Image" , fontsize=12)
                axs[0].axis('off')

                # Grad-CAM++ for Predicted Class
                axs[1].imshow(visualization_predicted)
                axs[1].set_title(f"Grad-CAM++ Pred Label: {predicted_class_name}", fontsize=12)
                axs[1].axis('off')

                # Grad-CAM++ for True Class
                axs[2].imshow(visualization_true)
                axs[2].set_title(f"Grad-CAM++ True Label: {folder}", fontsize=12)
                axs[2].axis('off')

                # Contrastive Heatmap
                axs[3].imshow(visualization_contrastive)
                axs[3].set_title("GradCAMs Difference", fontsize=12)
                axs[3].axis('off')

                plt.show()


In [ ]:
contrastive_gradCAMplusplus(model_path, folder_path, class_names)

# ScoreCAM

In [ ]:
def compute_ScoreCAM(model_path, folder_path, class_names, max_scorecam_channels=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  

    for folder in class_names:
        train_folder_path = os.path.join(folder_path, folder)
        if not os.path.isdir(train_folder_path):
            continue

        images = os.listdir(train_folder_path)
        if len(images) > 0:
            img_path = os.path.join(train_folder_path, images[0])
            print(f"Processing {img_path}...")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')
            input_tensor = transform(img_pil).unsqueeze(0).to(device)

            model.eval()
            middle_layer_activations = None

            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output

            target_layer = model.features[-1]  
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            predicted_class = torch.argmax(output, dim=1).item()
            predicted_class_name = class_names[predicted_class]
            correct_status = "(Correct)" if predicted_class_name == folder else "(Incorrect)"
            print(f"Predicted class: {predicted_class_name}{correct_status}")

            # **Score-CAM Calculation**
            print("Computing Score-CAM...")
            activations = middle_layer_activations.detach().cpu().numpy()[0]  
            num_activations = min(activations.shape[0], max_scorecam_channels)  
            score_weights = []

            for i in range(num_activations):
                if i % 10 == 0:
                    print(f"Processing activation map {i}/{num_activations}")

                mask = activations[i, :, :]
                mask = np.maximum(mask, 0)  
                if np.max(mask) != 0:
                    mask /= np.max(mask)  

                mask_resized = cv2.resize(mask, (input_tensor.shape[2], input_tensor.shape[3]))
                mask_tensor = torch.tensor(mask_resized, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

                perturbed_image = input_tensor * mask_tensor
                perturbed_output = model(perturbed_image)

                score_weights.append(perturbed_output[0, predicted_class].item())

            score_weights = np.array(score_weights)
            if np.max(score_weights) != 0:
                score_weights /= np.max(score_weights)

            scorecam_heatmap = np.zeros_like(activations[0])
            for i in range(num_activations):
                scorecam_heatmap += activations[i] * score_weights[i]

            scorecam_heatmap = np.maximum(scorecam_heatmap, 0)
            if np.max(scorecam_heatmap) != 0:
                scorecam_heatmap /= np.max(scorecam_heatmap)
            scorecam_heatmap_resized = cv2.resize(scorecam_heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

            # Visualizations
            img = np.array(img_pil, dtype=np.float32) / 255.0  
            scorecam_visualization = show_cam_on_image(img, scorecam_heatmap_resized, intensity=0.5)

            # Displaying Original Image and Score-CAM Visualization
            fig, axs = plt.subplots(1, 2, figsize=(10, 5))

            axs[0].imshow(np.array(img_pil))  
            axs[0].set_title(f"Original Image ({folder})" , fontsize=14)
            axs[0].axis('off')

            axs[1].imshow(scorecam_visualization)
            axs[1].set_title(f"Score-CAM: {predicted_class_name} {correct_status}", fontsize=14)
            axs[1].axis('off')


            plt.show()
    


In [ ]:
compute_ScoreCAM(model_path, folder_path, class_names, max_scorecam_channels=50)

## Contrastive ScoreCAM

In [ ]:
def contrastive_ScoreCAM(model_path, folder_path, class_names, max_scorecam_channels=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  

    for folder in class_names:
        train_folder_path = os.path.join(folder_path, folder)
        if not os.path.isdir(train_folder_path):
            continue

        images = os.listdir(train_folder_path)
        if len(images) > 0:
            img_path = os.path.join(train_folder_path, images[0])
            print(f"Processing {img_path}...")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')
            input_tensor = transform(img_pil).unsqueeze(0).to(device)

            model.eval()
            middle_layer_activations = None

            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output

            target_layer = model.features[-1]  
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            predicted_class = torch.argmax(output, dim=1).item()
            predicted_class_name = class_names[predicted_class]
            if predicted_class_name == folder:
                print(f"Predicted class: {predicted_class_name}")
            # Only proceed if predicted class is not equal to true class
            if predicted_class_name != folder:
                correct_status = "(Incorrect)"
                print(f"Predicted class: {predicted_class_name}{correct_status}")

                # Score-CAM Calculation
                print("Computing Score-CAM for predicted label...")
                activations = middle_layer_activations.detach().cpu().numpy()[0]  
                num_activations = min(activations.shape[0], max_scorecam_channels)  
                score_weights = []

                for i in range(num_activations):
                    if i % 10 == 0:
                        print(f"Processing activation map {i}/{num_activations}")

                    mask = activations[i, :, :]
                    mask = np.maximum(mask, 0)  
                    if np.max(mask) != 0:
                        mask /= np.max(mask)  

                    mask_resized = cv2.resize(mask, (input_tensor.shape[2], input_tensor.shape[3]))
                    mask_tensor = torch.tensor(mask_resized, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

                    perturbed_image = input_tensor * mask_tensor
                    perturbed_output = model(perturbed_image)

                    score_weights.append(perturbed_output[0, predicted_class].item())

                score_weights = np.array(score_weights)
                if np.max(score_weights) != 0:
                    score_weights /= np.max(score_weights)

                scorecam_heatmap_predicted = np.zeros_like(activations[0])
                for i in range(num_activations):
                    scorecam_heatmap_predicted += activations[i] * score_weights[i]

                scorecam_heatmap_predicted = np.maximum(scorecam_heatmap_predicted, 0)
                if np.max(scorecam_heatmap_predicted) != 0:
                    scorecam_heatmap_predicted /= np.max(scorecam_heatmap_predicted)
                scorecam_heatmap_predicted_resized = cv2.resize(scorecam_heatmap_predicted, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

                # Score-CAM Calculation for True Label
                print("[INFO] Computing Score-CAM for true label...")
                true_label_class = class_names.index(folder)  # Get true label index
                score_weights_true = []

                for i in range(num_activations):
                    if i % 10 == 0:
                        print(f"Processing activation map {i}/{num_activations}")

                    mask = activations[i, :, :]
                    mask = np.maximum(mask, 0)  
                    if np.max(mask) != 0:
                        mask /= np.max(mask)  

                    mask_resized = cv2.resize(mask, (input_tensor.shape[2], input_tensor.shape[3]))
                    mask_tensor = torch.tensor(mask_resized, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

                    perturbed_image = input_tensor * mask_tensor
                    perturbed_output = model(perturbed_image)

                    score_weights_true.append(perturbed_output[0, true_label_class].item())

                score_weights_true = np.array(score_weights_true)
                if np.max(score_weights_true) != 0:
                    score_weights_true /= np.max(score_weights_true)

                scorecam_heatmap_true = np.zeros_like(activations[0])
                for i in range(num_activations):
                    scorecam_heatmap_true += activations[i] * score_weights_true[i]

                scorecam_heatmap_true = np.maximum(scorecam_heatmap_true, 0)
                if np.max(scorecam_heatmap_true) != 0:
                    scorecam_heatmap_true /= np.max(scorecam_heatmap_true)
                scorecam_heatmap_true_resized = cv2.resize(scorecam_heatmap_true, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

                # Contrastive Heatmap (Difference between Predicted and True Label Heatmaps)
                print("Computing Contrastive Heatmap...")
                contrastive_heatmap = np.abs(scorecam_heatmap_predicted_resized - scorecam_heatmap_true_resized)

                # Visualizations
                img = np.array(img_pil, dtype=np.float32) / 255.0  
                scorecam_visualization_predicted = show_cam_on_image(img, scorecam_heatmap_predicted_resized, intensity=0.5)
                scorecam_visualization_true = show_cam_on_image(img, scorecam_heatmap_true_resized, intensity=0.5)
                scorecam_visualization_contrastive = show_cam_on_image(img, contrastive_heatmap, intensity=0.5)

                # Display Original Image and All Heatmaps
                fig, axs = plt.subplots(1, 4, figsize=(20, 5))

                axs[0].imshow(np.array(img_pil))  
                axs[0].set_title("Original Image", fontsize=12)
                axs[0].axis('off')

                axs[1].imshow(scorecam_visualization_predicted)
                axs[1].set_title(f"ScoreCAM Pred Label: {predicted_class_name}", fontsize=12)
                axs[1].axis('off')

                axs[2].imshow(scorecam_visualization_true)
                axs[2].set_title(f"ScoreCAM True Label: {folder}", fontsize=12)
                axs[2].axis('off')

                axs[3].imshow(scorecam_visualization_contrastive)
                axs[3].set_title("ScoreCAMs Difference", fontsize=12)
                axs[3].axis('off')

                plt.show()
            


In [ ]:
contrastive_ScoreCAM(model_path, folder_path, class_names, max_scorecam_channels=50)

# Combining Methods (GradCAM, GradCAM++, ScoreCAM)

In [ ]:
def compute_combined(model_path, folder_path, class_names, max_scorecam_channels=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
    ])
    
    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  

    for folder in class_names:
        train_folder_path = os.path.join(folder_path, folder)
        if not os.path.isdir(train_folder_path):
            continue

        images = os.listdir(train_folder_path)
        if len(images) > 0:
            img_path = os.path.join(train_folder_path, images[0])
            print(f"Processing {img_path}...")
            print('Actual class:', folder)

            img_pil = Image.open(img_path).convert('RGB')
            input_tensor = transform(img_pil).unsqueeze(0).to(device)

            model.eval()
            middle_layer_activations = None

            def hook_fn(module, input, output):
                nonlocal middle_layer_activations
                middle_layer_activations = output
                middle_layer_activations.requires_grad_(True)
                middle_layer_activations.retain_grad()

            target_layer = model.features[-1]  
            hook = target_layer.register_forward_hook(hook_fn)
            output = model(input_tensor)

            predicted_class = torch.argmax(output, dim=1).item()
            predicted_class_name = class_names[predicted_class]
            correct_status = "(Correct)" if predicted_class_name == folder else "(Incorrect)"
            print(f"Predicted class: {predicted_class_name} {correct_status}")

            # Grad-CAM Calculation
            print("Computing Grad-CAM...")
            output[0, predicted_class].backward(retain_graph=True)
            gradients = middle_layer_activations.grad.data
            pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
            activations = middle_layer_activations.detach().cpu().numpy()[0]

            for i in range(activations.shape[0]):
                activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
            
            gradcam_heatmap = np.mean(activations, axis=0)
            gradcam_heatmap = np.maximum(gradcam_heatmap, 0)
            if np.max(gradcam_heatmap) != 0:
                gradcam_heatmap /= np.max(gradcam_heatmap)
            gradcam_heatmap_resized = cv2.resize(gradcam_heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

            # Grad-CAM++ Calculation
            print("Computing Grad-CAM++...")
            grads_squared = gradients ** 2
            grads_third = grads_squared * gradients
            sum_activations = torch.sum(middle_layer_activations, dim=[2, 3], keepdim=True)

            alpha_numer = grads_squared
            alpha_denom = 2 * grads_squared + sum_activations * grads_third
            alpha = alpha_numer / (alpha_denom + 1e-10)
            weights = torch.sum(alpha * torch.relu(gradients), dim=[0, 2, 3])
            
            activations = middle_layer_activations.detach().cpu().numpy()[0]
            for i in range(activations.shape[0]):
                activations[i, :, :] *= weights[i].detach().cpu().numpy()

            gradcampp_heatmap = np.mean(activations, axis=0)
            gradcampp_heatmap = np.maximum(gradcampp_heatmap, 0)
            if np.max(gradcampp_heatmap) != 0:
                gradcampp_heatmap /= np.max(gradcampp_heatmap)
            gradcampp_heatmap_resized = cv2.resize(gradcampp_heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

            # Score-CAM Calculation
            print("Computing Score-CAM...")
            activations = middle_layer_activations.detach().cpu().numpy()[0]  
            num_activations = min(activations.shape[0], max_scorecam_channels)  
            score_weights = []

            for i in range(num_activations):
                if i % 10 == 0:
                    print(f"Processing activation map {i}/{num_activations}")

                mask = activations[i, :, :]
                mask = np.maximum(mask, 0)  
                if np.max(mask) != 0:
                    mask /= np.max(mask)  

                mask_resized = cv2.resize(mask, (input_tensor.shape[2], input_tensor.shape[3]))
                mask_tensor = torch.tensor(mask_resized, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

                perturbed_image = input_tensor * mask_tensor
                perturbed_output = model(perturbed_image)

                score_weights.append(perturbed_output[0, predicted_class].item())

            score_weights = np.array(score_weights)
            if np.max(score_weights) != 0:
                score_weights /= np.max(score_weights)

            scorecam_heatmap = np.zeros_like(activations[0])
            for i in range(num_activations):
                scorecam_heatmap += activations[i] * score_weights[i]

            scorecam_heatmap = np.maximum(scorecam_heatmap, 0)
            if np.max(scorecam_heatmap) != 0:
                scorecam_heatmap /= np.max(scorecam_heatmap)
            scorecam_heatmap_resized = cv2.resize(scorecam_heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

            # Visualisations
            img = np.array(img_pil, dtype=np.float32) / 255.0  
            gradcam_visualization = show_cam_on_image(img, gradcam_heatmap_resized, intensity=0.5)
            gradcampp_visualization = show_cam_on_image(img, gradcampp_heatmap_resized, intensity=0.5)
            scorecam_visualization = show_cam_on_image(img, scorecam_heatmap_resized, intensity=0.5)

            # Displaying All Four Images Side by Side
            fig, axs = plt.subplots(1, 4, figsize=(20, 5))

            axs[0].imshow(np.array(img_pil))  
            axs[0].set_title(f"Original Image ({folder})", fontsize=12)
            axs[0].axis('off')

            axs[1].imshow(gradcam_visualization)
            axs[1].set_title(f"Grad-CAM: {predicted_class_name} {correct_status}", fontsize=12)
            axs[1].axis('off')

            axs[2].imshow(gradcampp_visualization)
            axs[2].set_title(f"Grad-CAM++: {predicted_class_name} {correct_status}", fontsize=12)
            axs[2].axis('off')

            axs[3].imshow(scorecam_visualization)
            axs[3].set_title(f"Score-CAM: {predicted_class_name} {correct_status}", fontsize=12)
            axs[3].axis('off')

            plt.show()
        


In [ ]:
compute_combined(model_path, folder_path, class_names, max_scorecam_channels=50)

# Confidence

## Finding highest and lowest confident correct predictions

In [ ]:
def find_most_and_least_confident_images(model, folder_path, class_names, device):
    # Define the transformation
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
    ])
    
    most_confident = {"image_path": "", "confidence": -1, "predicted_class": ""}
    least_confident = {"image_path": "", "confidence": float('inf'), "predicted_class": ""}

    # Iterating through the folder and process each image using tqdm for the progress bar
    images = os.listdir(folder_path)
    for img_name in tqdm(images, desc="Processing images"):
        img_path = os.path.join(folder_path, img_name)
        if not os.path.isfile(img_path):  # Skip non-file items (like directories)
            continue

        img_pil = Image.open(img_path).convert('RGB')
        input_tensor = transform(img_pil).unsqueeze(0).to(device)

        # Getting the model output
        model.eval()
        output = model(input_tensor)

        # Getting predicted class
        predicted_class = torch.argmax(output, dim=1).item()
        predicted_class_name = class_names[predicted_class]

        # Getting the confidence score
        confidence = torch.nn.functional.softmax(output, dim=1)[0, predicted_class].item()

        # Checking if the prediction is correct
        actual_class_name = os.path.basename(folder_path) 
        if predicted_class_name == actual_class_name:
            # If correct prediction, update most and least confident
            if confidence > most_confident["confidence"]:
                most_confident["image_path"] = img_path
                most_confident["confidence"] = confidence
                most_confident["predicted_class"] = predicted_class_name

            if confidence < least_confident["confidence"]:
                least_confident["image_path"] = img_path
                least_confident["confidence"] = confidence
                least_confident["predicted_class"] = predicted_class_name

    return most_confident, least_confident


# Example usage:
model_path = "model_b7_2_epoch_7_3.pth"
model = torch.load(model_path, weights_only = False)
model = model.to(device)

# Loading model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path, weights_only=False)
model = model.to(device)

## MEL

In [ ]:
# Finding the most and least confident correct predictions
most_confident_mel, least_confident_mel = find_most_and_least_confident_images(model, '/Users/inescocco/Desktop/ISIC2019/Test_sorted/MEL', class_names, device)

print("\nMost Confident Correct Prediction:")
print(f"Image: {most_confident_mel['image_path']}, Confidence: {most_confident_mel['confidence']}, Predicted Class: {most_confident_mel['predicted_class']}")

print("\nLeast Confident Correct Prediction:")
print(f"Image: {least_confident_mel['image_path']}, Confidence: {least_confident_mel['confidence']}, Predicted Class: {least_confident_mel['predicted_class']}")


## BCC

In [ ]:
# Finding the most and least confident correct predictions
most_confident_bcc, least_confident_bcc = find_most_and_least_confident_images(model, '/Users/inescocco/Desktop/ISIC2019/Test_sorted/BCC', class_names, device)

print("\nMost Confident Correct Prediction:")
print(f"Image: {most_confident_bcc['image_path']}, Confidence: {most_confident_bcc['confidence']}, Predicted Class: {most_confident_bcc['predicted_class']}")

print("\nLeast Confident Correct Prediction:")
print(f"Image: {least_confident_bcc['image_path']}, Confidence: {least_confident_bcc['confidence']}, Predicted Class: {least_confident_bcc['predicted_class']}")


## SCC

In [ ]:
# Finding the most and least confident correct predictions
most_confident_scc, least_confident_scc = find_most_and_least_confident_images(model, '/Users/inescocco/Desktop/ISIC2019/Test_sorted/SCC', class_names, device)

print("\nMost Confident Correct Prediction:")
print(f"Image: {most_confident_scc['image_path']}, Confidence: {most_confident_scc['confidence']}, Predicted Class: {most_confident_bcc['predicted_class']}")

print("\nLeast Confident Correct Prediction:")
print(f"Image: {least_confident_scc['image_path']}, Confidence: {least_confident_scc['confidence']}, Predicted Class: {least_confident_bcc['predicted_class']}")


## Printing heatmaps for these images

In [ ]:
def compute_combined(model_path, folder_path, class_names, most_confident, least_confident, max_scorecam_channels=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False)
    model = model.to(device)
    model.eval()  # Set model to evaluation mode

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
    ])

    def show_cam_on_image(img, mask, intensity=0.6):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  
        cam = heatmap * intensity + img * (1 - intensity)
        return np.uint8(255 * cam)  

    for img_path in [most_confident['image_path'], least_confident['image_path']]:
        if not os.path.isfile(img_path):
            continue

        img_pil = Image.open(img_path).convert('RGB')
        input_tensor = transform(img_pil).unsqueeze(0).to(device)

        # Using the stored predicted class instead of re-predicting
        if img_path == most_confident['image_path']:
            predicted_class_name = most_confident['predicted_class']
        elif img_path == least_confident['image_path']:
            predicted_class_name = least_confident['predicted_class']
        else:
            raise ValueError("Image path does not match most or least confident images")

        middle_layer_activations = None

        def hook_fn(module, input, output):
            nonlocal middle_layer_activations
            middle_layer_activations = output
            middle_layer_activations.requires_grad_(True)
            middle_layer_activations.retain_grad()

        target_layer = model.features[-1] 
        hook = target_layer.register_forward_hook(hook_fn)
        
        output = model(input_tensor)
        predicted_class = class_names.index(predicted_class_name) 

        # Grad-CAM Calculation
        output[0, predicted_class].backward(retain_graph=True)
        gradients = middle_layer_activations.grad.data
        pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
        activations = middle_layer_activations.detach().cpu().numpy()[0]

        for i in range(activations.shape[0]):
            activations[i, :, :] *= pooled_gradients[i].cpu().numpy()
        
        gradcam_heatmap = np.mean(activations, axis=0)
        gradcam_heatmap = np.maximum(gradcam_heatmap, 0)
        if np.max(gradcam_heatmap) != 0:
            gradcam_heatmap /= np.max(gradcam_heatmap)
        gradcam_heatmap_resized = cv2.resize(gradcam_heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

        # Grad-CAM++ Calculation
        grads_squared = gradients ** 2
        grads_third = grads_squared * gradients
        sum_activations = torch.sum(middle_layer_activations, dim=[2, 3], keepdim=True)

        alpha_numer = grads_squared
        alpha_denom = 2 * grads_squared + sum_activations * grads_third
        alpha = alpha_numer / (alpha_denom + 1e-10)
        weights = torch.sum(alpha * torch.relu(gradients), dim=[0, 2, 3])
        
        activations = middle_layer_activations.detach().cpu().numpy()[0]
        for i in range(activations.shape[0]):
            activations[i, :, :] *= weights[i].detach().cpu().numpy()

        gradcampp_heatmap = np.mean(activations, axis=0)
        gradcampp_heatmap = np.maximum(gradcampp_heatmap, 0)
        if np.max(gradcampp_heatmap) != 0:
            gradcampp_heatmap /= np.max(gradcampp_heatmap)
        gradcampp_heatmap_resized = cv2.resize(gradcampp_heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

        # Score-CAM Calculation
        activations = middle_layer_activations.detach().cpu().numpy()[0]  
        num_activations = min(activations.shape[0], max_scorecam_channels)  
        score_weights = []

        for i in range(num_activations):
            mask = activations[i, :, :]
            mask = np.maximum(mask, 0)  
            if np.max(mask) != 0:
                mask /= np.max(mask)  

            mask_resized = cv2.resize(mask, (input_tensor.shape[2], input_tensor.shape[3]))
            mask_tensor = torch.tensor(mask_resized, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

            perturbed_image = input_tensor * mask_tensor
            perturbed_output = model(perturbed_image)

            score_weights.append(perturbed_output[0, predicted_class].item())

        score_weights = np.array(score_weights)
        if np.max(score_weights) != 0:
            score_weights /= np.max(score_weights)

        scorecam_heatmap = np.zeros_like(activations[0])
        for i in range(num_activations):
            scorecam_heatmap += activations[i] * score_weights[i]

        scorecam_heatmap = np.maximum(scorecam_heatmap, 0)
        if np.max(scorecam_heatmap) != 0:
            scorecam_heatmap /= np.max(scorecam_heatmap)
        scorecam_heatmap_resized = cv2.resize(scorecam_heatmap, (img_pil.size[0], img_pil.size[1]), interpolation=cv2.INTER_LINEAR)

        # Visualisations
        img = np.array(img_pil, dtype=np.float32) / 255.0  
        gradcam_visualization = show_cam_on_image(img, gradcam_heatmap_resized, intensity=0.5)
        gradcampp_visualization = show_cam_on_image(img, gradcampp_heatmap_resized, intensity=0.5)
        scorecam_visualization = show_cam_on_image(img, scorecam_heatmap_resized, intensity=0.5)

        # Displaying All Four Images Side by Side
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))

        axs[0].imshow(np.array(img_pil))  
        image_type = "Most Confident" if img_path == most_confident['image_path'] else "Least Confident"
        axs[0].set_title(f"{image_type} - Original Image ({predicted_class_name})", fontsize=18)
        axs[0].axis('off')

        axs[1].imshow(gradcam_visualization)
        axs[1].set_title(f"Grad-CAM: {predicted_class_name}", fontsize=18)
        axs[1].axis('off')

        axs[2].imshow(gradcampp_visualization)
        axs[2].set_title(f"Grad-CAM++: {predicted_class_name}", fontsize=18)
        axs[2].axis('off')

        axs[3].imshow(scorecam_visualization)
        axs[3].set_title(f"Score-CAM: {predicted_class_name}", fontsize=18)
        axs[3].axis('off')

        plt.show()


In [ ]:
compute_combined(model_path, '/Users/inescocco/Desktop/ISIC2019/Test_sorted/MEL', class_names, most_confident_mel, least_confident_mel, max_scorecam_channels=50)

In [ ]:
compute_combined(model_path, '/Users/inescocco/Desktop/ISIC2019/Test_sorted/BCC', class_names, most_confident_bcc, least_confident_bcc, max_scorecam_channels=50)


In [ ]:
compute_combined(model_path, '/Users/inescocco/Desktop/ISIC2019/Test_sorted/SCC', class_names, most_confident_scc, least_confident_scc, max_scorecam_channels=50)
